In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import re
import time
import os
from selenium import webdriver

In [3]:
def random_user_agent():
    try:
        ua = UserAgent()
        return ua.random
    except:
        default_ua = 'Mozilla/5.0 (Macintosh; Intel Mac O21S X 10_12_3) \
                AppleWebKit/537.36 (KHTML, like Gecko) \
                Chrome/58.0.3029.110 Safari/537.36'
        return default_ua

def get_soup(url):
    
    headers = {'User-Agent': random_user_agent()}
    response = requests.get(url, headers=headers)
    results = response.content
    if not response.status_code == 404:
        soup = BeautifulSoup(results, 'lxml')
    return soup

def soup_attempts(url, total_attempts=5):

    soup = get_soup(url)

    if soup:
        return soup
    else:
        attempts = 0
        while attempts < total_attempts:
            time.sleep(3)
            soup = get_soup(url)
            if soup:
                return soup
        raise ValueError(f'FAILED to get soup for apt url {url}')

In [4]:
url = 'https://www.corcoran.com/nyc-real-estate/for-sale/williamsburg/30-bartlett-street/5151565'
soup = soup_attempts(url)

In [5]:
soup.find('figure', class_='getCarouselSlides__SlideFigure-cel6pe-2 jpnhrN')

<figure class="getCarouselSlides__SlideFigure-cel6pe-2 jpnhrN"><img alt="slide image" class="getCarouselSlides__SlideImage-cel6pe-3 cNdyRy" src="https://mediarouting.vestahub.com/Media/68020238/box/800x800"/></figure>

In [6]:
soup.find('div', class_='carousel-item')

<div class="carousel-item active"><div aria-label="slide image" aria-pressed="false" class="getCarouselSlides__SlideImageWrapper-cel6pe-1 kRBdIV" role="button" tabindex="0"><figure class="getCarouselSlides__SlideFigure-cel6pe-2 jpnhrN"><img alt="slide image" class="getCarouselSlides__SlideImage-cel6pe-3 cNdyRy" src="https://mediarouting.vestahub.com/Media/68020238/box/800x800"/></figure></div><div class="carousel-caption d-none d-md-block"><h3></h3><p>caption</p></div></div>

In [7]:
soup.find('figure', class_='getCarouselSlides__SlideFigure-cel6pe-2 jpnhrN')

<figure class="getCarouselSlides__SlideFigure-cel6pe-2 jpnhrN"><img alt="slide image" class="getCarouselSlides__SlideImage-cel6pe-3 cNdyRy" src="https://mediarouting.vestahub.com/Media/68020238/box/800x800"/></figure>

In [8]:
soup.find('div', class_='getCarouselSlides__CarouselTitle-cel6pe-0 iWXEDU')